# Auxiliary functions to load CIFAR dataset

In [2]:
import random
import numpy as np
import os
import platform
from six.moves import cPickle as pickle
import matplotlib.pyplot as plt

def load_CIFAR10(ROOT):
    """ load all of cifar """
    xs = []
    ys = []
    for b in range(1, 6):
        f = os.path.join(ROOT, "data_batch_%d" % (b,))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, "test_batch"))
    return Xtr, Ytr, Xte, Yte

def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, "rb") as f:
        datadict = load_pickle(f)
        X = datadict["data"]
        Y = datadict["labels"]
        X = X.reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1).astype("float")
        Y = np.array(Y)
        return X, Y
    
def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == "2":
        return pickle.load(f)
    elif version[0] == "3":
        return pickle.load(f, encoding="latin1")
    raise ValueError("invalid python version: {}".format(version))

In [3]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Load CIFAR dataset 

In [5]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs/datasets/cifar-10-batches-py'
    
    # Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
    try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


# SOFTMAX

In [6]:
import time

def softmax_loss_naive(W, X, y, reg):
    """
    Loss funtion: implementation with loops

    There are C classes, we operate on minibatches of N examples, and inputs have dimension D

    Inputs:
    - W: A numpy array of shape (D, C) containing weights.
    - X: A numpy array of shape (N, D) containing a minibatch of data.
    - y: A numpy array of shape (N,) containing training labels; y[i] = c means
      that X[i] has label c, where 0 <= c < C.
    - reg: (float) regularization strength

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W; an array of same shape as W
    """

    loss = 0.0
    dW = np.zeros_like(W)

    num_train = X.shape[0]
    num_dims = X.shape[1]
    num_classes = W.shape[1]
    loss = 0

    for i in range(num_train):
        fs = np.zeros(num_classes)
        for j in range(num_classes):
            fs[j] = X[i] @ W[:, j]
        fs = fs - fs.max() #for numerical stability of exponentiation/division
        probs = np.exp(fs)/np.sum(np.exp(fs))
        loss += -np.log(probs[y[i]])
        
        probs[y[i]] -= 1     
        dW += np.outer(X[i, :], probs)

    loss /= num_train
    loss += reg*np.sum(W**2)

    dW /= num_train
    dW += reg*2*W 

    return loss, dW

In [7]:
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.310664
sanity check: 2.302585


In [10]:
from random import randrange
def grad_check_sparse(f, x, analytic_grad, num_checks=10, h=1e-5):
    """
    sample a few random elements and only return numerical in this dimensions.
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )

In [11]:
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# The numeric gradient should be close to the analytic gradient.
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# Another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 0.340114 analytic: 0.340114, relative error: 1.022008e-07
numerical: -2.031211 analytic: -2.031211, relative error: 9.128577e-09
numerical: 0.226404 analytic: 0.226404, relative error: 1.837265e-08
numerical: 0.106643 analytic: 0.106643, relative error: 1.283587e-07
numerical: -2.948276 analytic: -2.948276, relative error: 1.391475e-08
numerical: 0.672194 analytic: 0.672194, relative error: 1.340894e-08
numerical: -1.495760 analytic: -1.495760, relative error: 1.362676e-08
numerical: -2.486766 analytic: -2.486766, relative error: 5.193148e-09
numerical: 2.080274 analytic: 2.080273, relative error: 4.061204e-08
numerical: 0.676472 analytic: 0.676472, relative error: 4.345756e-08
numerical: -1.133299 analytic: -1.133300, relative error: 9.095495e-08
numerical: -0.655991 analytic: -0.655991, relative error: 1.122232e-08
numerical: -0.658423 analytic: -0.658423, relative error: 8.635434e-09
numerical: -0.309786 analytic: -0.309786, relative error: 1.224031e-07
numerical: 1.70465

### Vectorized version

In [12]:
def softmax_loss_vectorized(W, X, y, reg):
    """
    Softmax loss function, vectorized version.

    Inputs and outputs are the same as softmax_loss_naive.
    """
    loss = 0.0
    dW = np.zeros_like(W)

    num_train = X.shape[0]
    num_dims = X.shape[1]
    num_classes = W.shape[1]
    loss = 0
    fs = np.zeros((num_train, num_classes))     
    
    fs = X @ W
    fs = fs - fs.max(axis = 1, keepdims = True) #exponents that are numerically stable
    probs = np.exp(fs)/np.sum(np.exp(fs), axis = 1, keepdims = True)
    loss = -np.sum(np.log(probs[range(num_train), y]))

    probs[range(num_train), y] -= 1
    dW = np.dot(X.T, probs)

    loss = loss/num_train + reg*np.sum(W**2)
    dW /= num_train
    dW += reg*2*W 
    
    return loss, dW

In [14]:
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# Frobenius norm to compare the two versions of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.310664e+00 computed in 0.073085s
vectorized loss: 2.310664e+00 computed in 0.006444s
Loss difference: 0.000000
Gradient difference: 0.000000


## Hyperparameters tuning

In [15]:
class LinearClassifier(object):
    def __init__(self):
        self.W = None

    def train(
        self,
        X,
        y,
        learning_rate=1e-3,
        reg=1e-5,
        num_iters=100,
        batch_size=200,
        verbose=False,
    ):
        """
        Train this classifier using stochastic gradient descent.

        Inputs:
        - X: A numpy array of shape (N, D) containing training data; there are N
          training samples each of dimension D.
        - y: A numpy array of shape (N,) containing training labels; y[i] = c
          means that X[i] has label 0 <= c < C for C classes.
        - learning_rate: (float) learning rate for optimization.
        - reg: (float) regularization strength.
        - num_iters: (integer) number of steps to take when optimizing.
        - batch_size: (integer) number of training examples to use at each step.
        - verbose: (boolean) If true, print progress during optimization.

        Outputs:
        A list containing the value of the loss function at each training iteration.
        """
        num_train, dim = X.shape
        num_classes = (np.max(y) + 1)
        if self.W is None:
            self.W = 0.001 * np.random.randn(dim, num_classes)

        # Run stochastic gradient descent to optimize W
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None

            inds = np.random.choice(num_train, size = batch_size)
            X_batch = X[inds]
            y_batch = y[inds]

            loss, grad = self.loss(X_batch, y_batch, reg)
            loss_history.append(loss)

            self.W -= learning_rate*grad

            if verbose and it % 100 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

        return loss_history

    def predict(self, X):
        """
        Use the trained weights of this linear classifier to predict labels for data points.

        Inputs:
        - X: A numpy array of shape (N, D) containing training data; there are N training samples each of dimension D.

        Returns:
        - y_pred: Predicted labels for the data in X. 
            y_pred is a 1-dimensional array of length N, and each element is an integer giving the predicted class.
        """
        y_pred = np.zeros(X.shape[0])

        scores = X@self.W
        y_pred = np.argmax(scores, axis = 1)

        return y_pred

    def loss(self, X_batch, y_batch, reg):
        """
        Compute the loss function and its derivative.
        Subclasses will override this.

        Inputs:
        - X_batch: A numpy array of shape (N, D) containing a minibatch of N data points; each point has dimension D.
        - y_batch: A numpy array of shape (N,) containing labels for the minibatch.
        - reg: (float) regularization strength.

        Returns: A tuple containing:
        - loss as a single float
        - gradient with respect to self.W; an array of the same shape as W
        """
        pass

class Softmax(LinearClassifier):
    """ A subclass that uses the Softmax + Cross-entropy loss function """

    def loss(self, X_batch, y_batch, reg):
        return softmax_loss_vectorized(self.W, X_batch, y_batch, reg)

In [16]:
import itertools

results = {}
best_val = -1
best_softmax = None

learning_rates = np.linspace(1e-7, 1e-6, 5)
regularization_strengths = np.linspace(1e3, 1e4, 5)

for learning_rate, regularization_strength in itertools.product(learning_rates, regularization_strengths):

    smax = Softmax()
    loss = smax.train(X_train, y_train, learning_rate, regularization_strength,
                    num_iters=1000, verbose=True)
  
    y_train_pred = smax.predict(X_train)
    training_accuracy = np.mean(y_train == y_train_pred)

    y_val_pred = smax.predict(X_val)
    validation_accuracy = np.mean(y_val == y_val_pred)

    results[(learning_rate, regularization_strength)] = (training_accuracy, validation_accuracy)
    if validation_accuracy > best_val:
        best_val = validation_accuracy
        best_softmax = smax
    
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

iteration 0 / 1000: loss 37.032505
iteration 100 / 1000: loss 33.372618
iteration 200 / 1000: loss 31.967352
iteration 300 / 1000: loss 30.360112
iteration 400 / 1000: loss 28.766938
iteration 500 / 1000: loss 27.595878
iteration 600 / 1000: loss 26.611958
iteration 700 / 1000: loss 25.577488
iteration 800 / 1000: loss 24.639453
iteration 900 / 1000: loss 23.680559
iteration 0 / 1000: loss 106.323937
iteration 100 / 1000: loss 91.759710
iteration 200 / 1000: loss 80.632472
iteration 300 / 1000: loss 70.811538
iteration 400 / 1000: loss 62.116345
iteration 500 / 1000: loss 54.683418
iteration 600 / 1000: loss 48.137582
iteration 700 / 1000: loss 42.613545
iteration 800 / 1000: loss 37.473201
iteration 900 / 1000: loss 33.025555
iteration 0 / 1000: loss 172.950937
iteration 100 / 1000: loss 137.149036
iteration 200 / 1000: loss 109.942489
iteration 300 / 1000: loss 88.054601
iteration 400 / 1000: loss 70.901584
iteration 500 / 1000: loss 57.043413
iteration 600 / 1000: loss 46.183285
ite

iteration 500 / 1000: loss 1.929240
iteration 600 / 1000: loss 1.975377
iteration 700 / 1000: loss 1.933624
iteration 800 / 1000: loss 2.047878
iteration 900 / 1000: loss 1.980599
iteration 0 / 1000: loss 243.627364
iteration 100 / 1000: loss 12.288538
iteration 200 / 1000: loss 2.432172
iteration 300 / 1000: loss 2.009451
iteration 400 / 1000: loss 2.018159
iteration 500 / 1000: loss 2.069090
iteration 600 / 1000: loss 2.047536
iteration 700 / 1000: loss 1.989479
iteration 800 / 1000: loss 1.966102
iteration 900 / 1000: loss 2.008686
iteration 0 / 1000: loss 311.427726
iteration 100 / 1000: loss 7.333757
iteration 200 / 1000: loss 2.069608
iteration 300 / 1000: loss 1.995267
iteration 400 / 1000: loss 2.059871
iteration 500 / 1000: loss 1.964225
iteration 600 / 1000: loss 2.012957
iteration 700 / 1000: loss 2.031487
iteration 800 / 1000: loss 1.937864
iteration 900 / 1000: loss 2.005086
lr 1.000000e-07 reg 1.000000e+03 train accuracy: 0.243571 val accuracy: 0.228000
lr 1.000000e-07 re

In [17]:
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

softmax on raw pixels final test set accuracy: 0.382000
